In [2]:
import math
import ast
import pandas as pd

In [3]:
sorted_df = pd.read_excel("sortedmasterchem.xlsx")
sorted_df.drop('Unnamed: 0', axis=1, inplace=True)

# Convert Chemistry and Hate to python list instead of string
sorted_df['Chemistry'] = sorted_df['Chemistry'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sorted_df['Hate'] = sorted_df['Hate'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [4]:
# Scale dictionary values

In [5]:
def min_max_scale(dictionary):
    values = list(dictionary.values())
    min_val, max_val = min(values), max(values)
    
    if min_val == max_val:  # Prevent division by zero
        return {k: 0.50 for k in dictionary}  # Default to mid-range if all values are the same

    return {k: round((v - min_val) / (max_val - min_val), 2) for k, v in dictionary.items()}


In [6]:
# Chemistry metric function

In [7]:
import math

def calculate_chemistry_metric(current_team, drafted_players, df, k=0.9, x0=4.5, neg_chem_weight=.5):
    # Initialize a dictionary to store the chemistry metric for each available player
    chemistry_metrics = {}

    # Generate available players
    available_players = list(set(df['Character Name'].tolist()) - set(drafted_players))
    available_players = list(set(available_players) - set(current_team))

    # Calculate the weight based on the size of the current team
    team_size = len(current_team)
    weight_current_team = 1 / (1 + math.exp(-k * (team_size - x0)))
    weight_available_players = 1 - weight_current_team
    weight_unique_chem = weight_available_players
    print("team_size", team_size)
    print("weight_current_team:", weight_current_team)
    print("weight_available_players:", weight_available_players)

    for player in available_players:
        # Get the player's row from the dataframe
        player_row = df[df['Character Name'] == player].iloc[0]

        # Calculate positive and negative chemistry connections with the current team
        positive_chem_current_team = len(set(player_row['Chemistry']).intersection(current_team))
        negative_chem_current_team = len(set(player_row['Hate']).intersection(current_team)) * neg_chem_weight

        # Calculate positive and negative chemistry connections with the available players
        positive_chem_available_players = len(set(player_row['Chemistry']).intersection(available_players))
        negative_chem_available_players = len(set(player_row['Hate']).intersection(available_players)) * neg_chem_weight

        # Calculate the number of unique chemistry connections the player would add to the current team
        unique_chem = len(set(player_row['Chemistry']) - set([item for string in current_team for item in df.loc[df['Character Name'] == string, 'Chemistry'].sum()]))


        # Calculate the chemistry metric
        chemistry_metric = (
            weight_current_team * (positive_chem_current_team - negative_chem_current_team) +
            weight_available_players * (positive_chem_available_players - negative_chem_available_players) +
            weight_unique_chem * (unique_chem)
        )

        # Store the chemistry metric for the player
        chemistry_metrics[player] = round(chemistry_metric, 2)

    scaled_chemistry_metrics = min_max_scale(chemistry_metrics)

    return scaled_chemistry_metrics


In [8]:

current_team = ['Bowser']
available_players = ['Luigi', 'Red Mii Male']

metrics = calculate_chemistry_metric(current_team, available_players, sorted_df)
dict(sorted(metrics.items(), key=lambda item: item[1], reverse=True))

team_size 1
weight_current_team: 0.041091278200465015
weight_available_players: 0.958908721799535


{'Birdo': 1.0,
 'Baby Peach': 0.98,
 'Mario': 0.96,
 'Toadette': 0.84,
 'Toadsworth': 0.8,
 'Baby DK': 0.78,
 'Yellow Pianta': 0.75,
 'Pianta': 0.75,
 'Red Mii Female': 0.73,
 'Peach': 0.71,
 'Red Pianta': 0.71,
 'Baby Daisy': 0.67,
 'Green Mii Male': 0.65,
 'Green Mii Female': 0.65,
 'Monty Mole': 0.58,
 'Boo': 0.56,
 'Blue Mii Female': 0.55,
 'Blue Mii Male': 0.55,
 'Yellow Toad': 0.54,
 'Green Toad': 0.54,
 'Blue Toad': 0.54,
 'Purple Toad': 0.54,
 'Baby Luigi': 0.54,
 'Baby Mario': 0.51,
 'Brown Mii Male': 0.51,
 'Brown Mii Female': 0.51,
 'Toad': 0.51,
 'Yellow Mii Female': 0.47,
 'Yellow Mii Male': 0.47,
 'Pink Mii Female': 0.44,
 'Pink Mii Male': 0.44,
 'Koopa Troopa': 0.44,
 'Red Koopa Troopa': 0.4,
 'Yoshi': 0.35,
 'Pink Yoshi': 0.35,
 'Yellow Yoshi': 0.35,
 'Noki': 0.35,
 'Green Noki': 0.35,
 'Blue Yoshi': 0.35,
 'Funky Kong': 0.35,
 'Light Blue Yoshi': 0.35,
 'Tiny Kong': 0.33,
 'King K Rool': 0.33,
 'Red Noki': 0.31,
 'Red Yoshi': 0.31,
 'Wiggler': 0.29,
 'Donkey Kong': 0.2

In [9]:
current_team = ['Bowser', 'Baby Luigi', 'Green Mii Male', 'Koopa Troopa', 'Green Dry Bones', 'Green Toad', 'Light Blue Mii Male', "Green Mii Female"]
available_players = ['Luigi', 'Red Mii Male']

big_metrics = calculate_chemistry_metric(current_team, available_players, sorted_df)
dict(sorted(big_metrics.items(), key=lambda item: item[1], reverse=True))

team_size 8
weight_current_team: 0.958908721799535
weight_available_players: 0.04109127820046499


{'Koopa Paratroopa': 1.0,
 'Hammer Bro': 0.78,
 'Green Magikoopa': 0.66,
 'Red Koopa Troopa': 0.62,
 'Bowser Jr.': 0.59,
 'Red Koopa Paratroopa': 0.58,
 'Dry Bones': 0.57,
 'Blue Dry Bones': 0.57,
 'Dark Bones': 0.56,
 'Green Noki': 0.56,
 'Light Blue Yoshi': 0.56,
 'Kritter': 0.51,
 'Yellow Pianta': 0.48,
 'Pianta': 0.48,
 'King K Rool': 0.47,
 'Red Pianta': 0.47,
 'Baby Peach': 0.45,
 'Green Shy Guy': 0.43,
 'Toadette': 0.38,
 'Funky Kong': 0.38,
 'Tiny Kong': 0.37,
 'Boomerang Bro': 0.37,
 'Yoshi': 0.36,
 'Pink Yoshi': 0.36,
 'Yellow Yoshi': 0.36,
 'Peach': 0.36,
 'Blue Yoshi': 0.36,
 'Fire Bro': 0.36,
 'Toadsworth': 0.35,
 'Red Yoshi': 0.35,
 'Black Mii Female': 0.34,
 'Light Blue Mii Female': 0.34,
 'Black Mii Male': 0.34,
 'Baby DK': 0.29,
 'Birdo': 0.29,
 'Red Mii Female': 0.26,
 'Red Magikoopa': 0.25,
 'Yellow Magikoopa': 0.25,
 'Magikoopa': 0.25,
 'Baby Mario': 0.24,
 'Monty Mole': 0.23,
 'Goomba': 0.23,
 'Blue Mii Female': 0.22,
 'Blue Mii Male': 0.22,
 'Yellow Mii Female': 0

In [11]:
def create_player_tuples(scores):
    # Define the metrics to extract
    metrics = ['chem_score', 'slugging', 'charge_hit_power', 'slap_hit_power', 'speed', 'home_runs', 'pitching_stamina']

    player_tuples = []
    for player, stats in scores.items():
        # Calculate total score (sum of all metrics, modify as needed)
        total_score = sum(stats.get(metric, 0) for metric in metrics)

        # Extract values for required metrics, defaulting to 0 if missing
        values = [stats.get(metric, 0) for metric in metrics]

        # Create tuple: (player, total_score, metric values...)
        player_tuples.append((player, total_score, *values))

    return player_tuples

# Example dictionary
scores = {
    'Mario': {'chem_score': 0.8, 'slugging': 0.9, 'charge_hit_power': 70, 'slap_hit_power': 50, 'speed': 80, 'home_runs': 30, 'pitching_stamina': 40},
    'Luigi': {'chem_score': 0.5, 'slugging': 0.7, 'charge_hit_power': 60, 'slap_hit_power': 55, 'speed': 75, 'home_runs': 25, 'pitching_stamina': 45},
    'Peach': {'chem_score': 0.3, 'slugging': 0.6, 'charge_hit_power': 50, 'slap_hit_power': 60, 'speed': 90, 'home_runs': 20, 'pitching_stamina': 35},
}

# Generate tuples
player_tuples = create_player_tuples(scores)

# Print result
for tup in player_tuples:
    print(tup)


('Mario', 271.7, 0.8, 0.9, 70, 50, 80, 30, 40)
('Luigi', 261.2, 0.5, 0.7, 60, 55, 75, 25, 45)
('Peach', 255.9, 0.3, 0.6, 50, 60, 90, 20, 35)


In [14]:
for player, stats in scores.items():
    print(stats['chem_score'])

0.8
0.5
0.3
